In [ ]:
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time, math
from sklearn.metrics import r2_score
import time
start = time.time()

tf.set_random_seed(777)
np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='ann'
condition= 'release'   #release , #press
data_split_type= 'cycle_split'  #total_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for i in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,i), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,i), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,i), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,i), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)

        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')    
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)

        tf.reset_default_graph()

        X = tf.placeholder(tf.float32, shape=[None, 16])
        Y = tf.placeholder(tf.float32, shape=[None, 1])

        W1 = tf.get_variable("weight1", shape=[16, 64], initializer=tf.contrib.layers.xavier_initializer())
        W2 = tf.get_variable("weight2", shape=[64, 128], initializer=tf.contrib.layers.xavier_initializer())
        W3 = tf.get_variable("weight3", shape=[128, 1], initializer=tf.contrib.layers.xavier_initializer())

        b1 = tf.Variable(tf.random_normal([64]), name='bias1')
        b2 = tf.Variable(tf.random_normal([128]), name='bias2')
        b3 = tf.Variable(tf.random_normal([1]), name='bias3')

        L2 = tf.nn.relu(tf.matmul(X, W1) + b1)
        L3 = tf.nn.relu(tf.matmul(L2, W2) + b2)
        hypothesis = tf.matmul(L3, W3) + b3

        cost = tf.reduce_mean(tf.square(hypothesis - Y))
        train_op  = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

        from sklearn.metrics import r2_score
        from sklearn.metrics import explained_variance_score

        init = tf.global_variables_initializer()

        def run_train(sess, train_x, train_y):
            print("\nStart training")
            sess.run(init)
            batch_size = 10
            for epoch in range(300):
                total_batch = int(train_x.shape[0] / batch_size)
                for i in range(total_batch):
                    batch_x = train_x[i*batch_size:(i+1)*batch_size]
                    batch_y = train_y[i*batch_size:(i+1)*batch_size]
                    _, c = sess.run([train_op, cost], feed_dict={X: batch_x, Y: batch_y})
                print("Epoch #%d cost=%f" % (epoch, c))

        def cross_validate(sess, split_size=5):
            results = []
            kf = KFold(n_splits=split_size, shuffle=True)
            for train_idx, val_idx in kf.split(x_data_training, y_data_training):
                train_x = x_data_training[train_idx]
                train_y = y_data_training[train_idx]
                val_x = x_data_training[val_idx]
                val_y = y_data_training[val_idx]
                run_train(sess, train_x, train_y)
                results.append(sess.run(cost, feed_dict={X: val_x, Y: val_y}))
            return results

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            result = cross_validate(sess)
            result_mean=np.mean(result)
            cost_=sess.run(cost, feed_dict={X: x_data_test, Y: y_data_test})
            print("Cross-validation result: %s" % result)
            print("Test cost: %f" % sess.run(cost, feed_dict={X: x_data_test, Y: y_data_test}))
            y_predict_tr = sess.run(hypothesis, feed_dict={X: x_data_training})
            y_predict = sess.run(hypothesis, feed_dict={X: x_data_test})
            r2_score_tr= explained_variance_score(y_data_training, y_predict_tr)
            r2_score= r2_score(y_data_test, y_predict, multioutput='variance_weighted')
            saver = tf.train.Saver()
            saver.save(sess, 'save_path' %(model,data_split_type,condition,name,i))
            
            group_cost_tr.append(result_mean)
            group_cost.append(cost_)
            group_r2_score_tr.append(r2_score_tr)
            group_r2_score.append(r2_score)
            
        np.savetxt("save_path" %(model,data_split_type,condition,name,i),y_predict, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,i),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_predict)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,i), dpi=300)
    plt.show()        

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#ANN_regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.gaussian_process.kernels import Matern
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')

        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = LinearRegression(n_jobs=-1)
            regr.fit(x_data_training_,y_data_training_) 

            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')
        
        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#LinearRegression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.gaussian_process.kernels import Matern
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')        

        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = Ridge(random_state=0,alpha=0.001) 
            regr.fit(x_data_training_,y_data_training_)  

            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')
        
        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#Ridge_regressor

In [ ]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')

        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = linear_model.Lasso(random_state=0,alpha=0.00001) 
            regr.fit(x_data_training_,y_data_training_)   

            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j)) ##모델 저장
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#Lasso_regressor

In [ ]:
import numpy as np
import random
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneGroupOut
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = linear_model.LassoLars(alpha=0.0001)
            regr.fit(x_data_training_,y_data_training_)   
            
            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#LARS_regressor

In [ ]:
import numpy as np
import random
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import LeaveOneGroupOut
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = ElasticNet(random_state=0,l1_ratio=0.7, alpha=1e-08)
            regr.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#ENR

In [ ]:
import numpy as np
import random
from sklearn.kernel_ridge import KernelRidge
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

kernel = 1.0 * Matern(nu=0.5,length_scale=1)
start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            KRR_regr = KernelRidge(kernel=kernel, alpha=0.1)
            KRR_regr.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=KRR_regr.predict(x_data_training_)
            y_pred=KRR_regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=KRR_regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#KRR

In [ ]:
import numpy as np
import random
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import LeaveOneGroupOut
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            Bay_Reg =MultiOutputRegressor(linear_model.BayesianRidge(tol=0.1))
            Bay_Reg.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=Bay_Reg.predict(x_data_training_)
            y_pred=Bay_Reg.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=Bay_Reg.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#BRR

In [ ]:
import numpy as np
import random
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = MultiOutputRegressor(linear_model.ARDRegression(alpha_1=1e-04,alpha_2=1e-08,lambda_1=1e-08,lambda_2=1e-04))
            regr.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#ARD

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            forest_reg = RandomForestRegressor(max_features='sqrt', random_state=0, n_estimators=150, max_depth=15)
            forest_reg.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=forest_reg.predict(x_data_training_)
            y_pred=forest_reg.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=forest_reg.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#RF

In [ ]:
import numpy as np
import random
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            Ada_regr = MultiOutputRegressor(AdaBoostRegressor(random_state=0, loss='exponential',
                                                          learning_rate=0.1, n_estimators=100))
            Ada_regr.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=Ada_regr.predict(x_data_training_)
            y_pred=Ada_regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=Ada_regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#AdaBoost

In [ ]:
import numpy as np
import random
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            gradientboost_reg = MultiOutputRegressor(GradientBoostingRegressor(loss='ls',learning_rate=0.1, n_estimators=150,subsample=0.7, 
                                                   random_state=0))
            gradientboost_reg.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=gradientboost_reg.predict(x_data_training_)
            y_pred=gradientboost_reg.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=gradientboost_reg.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#Gradient Boost

In [ ]:
import numpy as np
import random
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            xg_reg =MultiOutputRegressor(xgb.XGBRegressor(learning_rate=0.1,subsample=0.7, max_depth=8, random_state=0))
            xg_reg.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=xg_reg.predict(x_data_training_)
            y_pred=xg_reg.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=xg_reg.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#XG Boost

In [ ]:
import numpy as np
import random
from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 
kernel = 1.0 * Matern(length_scale=1.0, nu=0.5)
start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups) 

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            svr_reg = svr_reg = MultiOutputRegressor(SVR(kernel=kernel, C=1))
            svr_reg.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=svr_reg.predict(x_data_training_)
            y_pred=svr_reg.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=svr_reg.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#SVM

In [ ]:
import numpy as np
import random
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            knn_reg = KNeighborsRegressor(weights='distance', n_neighbors= 13, p=1 )
            knn_reg.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=knn_reg.predict(x_data_training_)
            y_pred=knn_reg.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=knn_reg.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#KNN

In [ ]:
import numpy as np
import random
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import LeaveOneGroupOut
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='lasso' 
condition= 'press' #release
data_split_type= 'total_split' #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')
        
        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -4]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -4]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            regr = PLSRegression(n_components=1,tol=1e-08)
            regr.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=regr.predict(x_data_training_)
            y_pred=regr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_=regr.predict(x_data_test)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test,y_pred_)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#PLS

In [ ]:
import numpy as np
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import pickle
from joblib import dump, load 

kernel = 1.0 * Matern(length_scale=10.0, nu=0.5) 
start = time.time()

np.random.seed(999)

sector=["1","2","3","4","5","6","7","8","9"]
model='gpr' 
condition= 'release'    #release , #press
data_split_type= 'total_split'  #total_split, #cycle_split
group_cost_tr=[]
group_cost=[]
group_r2_score_tr=[]
group_r2_score=[]

for j in range(5,6):
    col=0
    row=2
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize = (8,8))
    
    for name in sector:
        r2_tr=0
        r2_val=0
        MSE_tr=0
        MSE_val=0
        train = []
        val = []
        train_p = []
        val_p = []
        
        if col > 2:
            col=0
            row-=1
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_tr = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        xy_te = np.loadtxt("Input_data_path" %(data_split_type,name,j), delimiter=',', dtype=np.float32)
        
        xy_data_tr=xy_tr.reshape(-1,21)
        xy_data_te=xy_te.reshape(-1,21)

        np.random.shuffle(xy_data_tr)
        np.random.shuffle(xy_data_te)
        
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_tr , delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), xy_data_te , delimiter=',',fmt='%4f')

        x_data_training = xy_data_tr[:, :-5]
        y_data_training = xy_data_tr[:, -5]
        x_data_test = xy_data_te[:, :-5]
        y_data_test = xy_data_te[:, -5]

        y_data_training = y_data_training.reshape(-1,1)
        y_data_test = y_data_test.reshape(-1,1)
        
        groups = np.zeros((len(xy_tr)))
        cut_data= int(len(xy_tr)/5)
        for i in range(int(len(groups)/cut_data)):
            groups[i*cut_data:(i*cut_data)+cut_data]=i
            
        logo = LeaveOneGroupOut()
        logo.get_n_splits(x_data_training, y_data_training, groups)

        logo.get_n_splits(groups=groups)  

        for train_index, val_index in logo.split(x_data_training, y_data_training, groups):

            x_data_training_, x_data_val = x_data_training[train_index], x_data_training[val_index]
            y_data_training_, y_data_val = y_data_training[train_index], y_data_training[val_index]
            
            gpr = GaussianProcessRegressor(random_state=0,alpha=1e-04,  kernel= kernel)
            gpr.fit(x_data_training_,y_data_training_) 
            
            y_pred_tr=gpr.predict(x_data_training_)
            y_pred=gpr.predict(x_data_val)
            
            r2_tr+=r2_score(y_data_training_, y_pred_tr, multioutput='variance_weighted')
            r2_val+=r2_score(y_data_val, y_pred, multioutput='variance_weighted')
            MSE_tr+=mean_squared_error(y_data_training_, y_pred_tr, multioutput='uniform_average')
            MSE_val+=mean_squared_error(y_data_val, y_pred, multioutput='uniform_average')

            train.append(y_data_training_)
            val.append(y_data_val)
            train_p.append(y_pred_tr)
            val_p.append(y_pred)
        
        dump(regr, 'save_path' %(model,data_split_type,condition,name,j))
        
        y_pred_, std_te=gpr.predict(x_data_test, return_std=True)
        std_te_=std_te.reshape(-1,1)
        r2_te_=r2_score(y_data_test, y_pred_, multioutput='variance_weighted')
        MSE_te_=mean_squared_error(y_data_test, y_pred_, multioutput='uniform_average')  
        y_pred_std_te_p = y_pred_+std_te_
        y_pred_std_te_m = y_pred_-std_te_
        group_cost_tr.append(MSE_tr/5)
        group_cost.append(MSE_te_)
        group_r2_score_tr.append(r2_tr/5)
        group_r2_score.append(r2_te_)
                
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_std_te_p, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j), y_pred_std_te_m, delimiter=',',fmt='%4f')
        np.savetxt("save_path" %(model,data_split_type,condition,name,j),y_data_test, delimiter=',',fmt='%4f')

        axes[row,col].plot([0,1])
        axes[row,col].scatter(y_data_test, y_pred_)
        axes[row,col].scatter(y_data_test, y_pred_std_te_p)
        axes[row,col].scatter(y_data_test, y_pred_std_te_m)
        col+=1
        
    fig.tight_layout()
    plt.savefig('save_path' %(model,data_split_type,condition,j), dpi=300)
    plt.show()

group_cost_tr=np.array(group_cost_tr).reshape(-1,1)
group_cost=np.array(group_cost).reshape(-1,1)
group_r2_score_tr=np.array(group_r2_score_tr).reshape(-1,1)
group_r2_score=np.array(group_r2_score).reshape(-1,1)
result_cost=np.concatenate((group_cost_tr,group_cost), axis=1)
result_r2_score=np.concatenate((group_r2_score_tr,group_r2_score), axis=1)
result_cost_r2_score=np.concatenate((result_cost,result_r2_score), axis=1)

np.savetxt("save_path" %(model,data_split_type,condition),result_cost_r2_score, delimiter=',',fmt='%4f')

print("time :", time.time() - start)

#GPR